In [15]:
import importlib
import sys
from tqdm import tqdm
import  os
import pickle
import glob
from datetime import datetime
import matplotlib.pyplot as plt
# Add the directories to the sys.path
dir_path = "C://Users//jony//Programming//Python//Anubis//anubis//" # insert your directory path
sys.path.append(dir_path + "Osiris//processing//python")
sys.path.append(dir_path + "tools")

import Analysis_tools as ATools
import Atlas_tools as AtTools
import proAnubis_Analysis_Tools
import Reconstruction_tools as RTools
import numpy as np
import mplhep as hep
import Timing_tools as TTools
import pickle
import tools.overview as overview
import rawFileReader
plt.rc('text', usetex=True)
import Visual_tools as VTools
hep.style.use([hep.style.ATLAS])

dir = "C://Users//jony//Programming//Python//Anubis//anubis//data//"

# Specify the directory


In [44]:
importlib.reload(overview)
file_name = "proAnubis_240815_1759.raw"
chunks, times, fReader = overview.get_chunks(file_name, max_process_event=1_000_000, saves = False)

Initial time 2024-08-15 16:59:49.312501 1723737589.312501


Processing Chunks proAnubis_240815_1759.raw: 100%|██████████| 10000/10000 [01:44<00:00, 95.65Chunks/s]


Ending time: 2024-08-15 17:03:04.904445
Number of chunks: 10000


In [ ]:
importlib.reload(proAnubis_Analysis_Tools)
TAnalyser = proAnubis_Analysis_Tools.Timing_Analyser(chunks[0],0)
for processedEvents, chunk in enumerate(chunks):
    TAnalyser.update_event(chunk, processedEvents)
    TAnalyser.read_tot_diffs()
    
tpt_mean, tot_std = TAnalyser.calculate_corrections()
with open(dir + "tpt_mean.pkl", "wb") as f:
    pickle.dump(tpt_mean, f)
with open(dir + "tot_std.pkl", "wb") as f:
    pickle.dump(tot_std, f)

In [4]:
with open(dir + "tpt_mean.pkl", "rb") as f:
    tpt_mean = pickle.load(f)
with open(dir + "tot_std.pkl", "rb") as f:
    tot_std = pickle.load(f)

In [46]:
importlib.reload(RTools)
reconstructor = RTools.Reconstructor(chunks[0], 0)
reconstructor.mean_tot = tpt_mean
reconstructor.std_tot = tot_std
interesting = []
vertices = []
distances = []
with tqdm(total=len(chunks)) as pbar:
    for processedEvents, chunk in enumerate(chunks):    
        reconstructor.update_event(chunk, processedEvents)
        clusters = reconstructor.cluster()
        chunk_tracks = reconstructor.reconstruct_tracks(clusters)
        for evt_num, event_tracks in enumerate(chunk_tracks):
            if len(event_tracks) > 1:
                distance, vertex = RTools.intersection(event_tracks[0], event_tracks[1])
                if vertex[2] < 0:
                    vertices.append(vertex)
                    distances.append(distance)
                interesting.append([processedEvents, evt_num])
                #interesting.append([processedEvents, evt_num, event_tracks])
        reconstructor.tracks = []
        pbar.update(1)



  0%|          | 0/10000 [00:00<?, ?it/s]

  9%|▉         | 927/10000 [00:21<02:51, 52.94it/s]C:\/Users//jony//Programming//Python//Anubis//anubis//tools\Reconstruction_tools.py:379: RuntimeWarning: invalid value encountered in divide
  normal = np.cross(a, b)/np.linalg.norm(np.cross(a,b)) #looking from b
 50%|█████     | 5016/10000 [01:56<01:39, 49.92it/s]C:\/Users//jony//Programming//Python//Anubis//anubis//tools\Reconstruction_tools.py:381: RuntimeWarning: invalid value encountered in divide
  a_parameter = np.cross(cb - ca + normal*closest_distance, b)/np.cross(a,b)
100%|██████████| 10000/10000 [03:59<00:00, 41.78it/s]


In [50]:
print(len(distances))

46


In [48]:
#3d scatter plot

points = np.array(vertices)
x = points[:, 0]
y = points[:, 1]
z = points[:, 2]

# Create a 3D scatter plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Scatter plot
ax.scatter(x, y, z, c='r', marker='o')

# Set labels
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
plt.show()

In [56]:
z = []
for i, vertex in enumerate(vertices):
    if distances[i] < 10:
        z.append(vertex[2])
hist, bins = np.histogram(z)
plt.step(bins[:-1], hist, where="mid")
plt.show()

In [43]:
importlib.reload(VTools)
importlib.reload(RTools)
reconstructor = RTools.Reconstructor(chunks[0], 0)
reconstructor.tot_mean = tpt_mean
reconstructor.tot_std = tot_std

tracks = []
#interesting = [(114, 39), (123, 65), (196, 92)]
for processedEvents, evt_num in interesting:
    reconstructor.tracks = []
    print(processedEvents, evt_num)
    reconstructor.update_event([chunks[processedEvents][evt_num]], processedEvents)
    clusters = reconstructor.cluster()
    chunk_tracks = reconstructor.reconstruct_tracks(clusters)
    tracks.append(chunk_tracks)
    VTools.event_3d_plot(chunks[processedEvents][evt_num], reconstructor, "Potential jets")

22 74
RPC: 4
Tracks: 2
114 39
RPC: 3
Tracks: 2
123 65
RPC: 4
Tracks: 2
138 7
RPC: 3
Tracks: 0
196 92
RPC: 3
Tracks: 2
221 61
RPC: 6
Tracks: 2
320 25
RPC: 4
Tracks: 1
322 65
RPC: 3
Tracks: 2
396 18
RPC: 4
Tracks: 2
455 60
RPC: 4
Tracks: 2
497 28
RPC: 4
Tracks: 2
556 89
RPC: 4
Tracks: 2
612 42
RPC: 3
Tracks: 2
928 92
RPC: 4
Tracks: 2


C:\/Users//jony//Programming//Python//Anubis//anubis//tools\Reconstruction_tools.py:379: RuntimeWarning: invalid value encountered in divide
  normal = np.cross(a, b)/np.linalg.norm(np.cross(a,b)) #looking from b


930 34
RPC: 4
Tracks: 2
956 11
RPC: 3
Tracks: 2
996 54
RPC: 4
Tracks: 2


In [34]:
RTools.intersection(tracks[0][0][0], tracks[0][0][1])

[-42.67013516 -42.67013516 -42.67013516]
[-42.67013516 -42.67013516 -42.67013516]
[ 99.74803065  33.22528422 -18.08087814]
[102.69959135  35.19438002 -13.90785517]


(np.float64(5.663621652675622),
 array([102.69959135,  35.19438002, -13.90785517]))

In [142]:
speeds = []
for tracks in reconstructor.tracks:
    if tracks:
        for track in tracks:
            speeds.append(track.speed)
print("Mean", np.mean(speeds))
hist, bins = np.histogram(speeds, bins=100, density=True)
plt.plot(bins[:-1], hist)
plt.vlines(np.mean(speeds), 0, max(hist), color="red", label=f"Mean {np.mean(speeds)} c")
plt.title("Speed of tracks")
plt.xlabel("Speed [c]")
plt.ylabel("Counts")
plt.legend()
plt.show()

Mean 1.0049729763710953


In [216]:
import scipy.optimize as opt
def gauss(x, a, mu, sigma):
    return a * np.exp(-(x - mu) ** 2 / (2 * sigma ** 2))

pairs = [[0,1], [0,2], [0,3], [0,4], [0,5]]
parameters1 = []
tof_hist = reconstructor.tof_histogram(pairs)
for idx, pair in enumerate(tof_hist):
    hist, bins = np.histogram(pair, bins = 50, density=True, range = (-20, 20))
    popt, pcov = opt.curve_fit(gauss, bins[:-1], hist, p0=[max(hist), 0, 10])
    parameters1.append([popt[1],abs(popt[2])])
    plt.plot(bins, gauss(bins, *popt), label = f"RPC pair {pairs[idx]} fit")    
    print("Mean", popt[1])
    print("sigma", popt[2])
    plt.plot(bins[:-1], hist, label = f"RPC pair {pairs[idx]}")
plt.xlabel("Time of flight [(25/32) ns]")
plt.ylabel("Normalized counts")
plt.title("Time of flight distribution")
plt.legend()
plt.show()
pairs = [[1,2], [1,3], [1,4], [1,5]]
parameters2 = []
tof_hist = reconstructor.tof_histogram(pairs)
for idx, pair in enumerate(tof_hist):
    hist, bins = np.histogram(pair, bins = 50, density=True, range = (-20, 20))
    popt, pcov = opt.curve_fit(gauss, bins[:-1], hist, p0=[max(hist), 0, 10])
    parameters2.append([popt[1], abs(popt[2])])
    plt.plot(bins, gauss(bins, *popt), label = f"RPC pair {pairs[idx]} fit")    
    print("Mean", popt[1])
    print("NEw mean", np.mean(pair))
    print("sigma", popt[2])
    plt.plot(bins[:-1], hist, label = f"RPC pair {pairs[idx]}")
plt.xlabel("Time of flight [(25/32) ns]")
plt.ylabel("Normalized counts")
plt.title("Time of flight distribution")
plt.legend()
plt.show()

Mean 0.25790911824685864
sigma -0.5818488944295358
Mean 0.06942782469835455
sigma -0.5634127422432911
Mean 2.17698945272121
sigma 2.5044894876283283
Mean 6.619902168271083
sigma 2.91238011053787
Mean 6.199631794023697
sigma 2.617768586336547
Mean -0.3896962620845962
NEw mean -0.2697363551278123
sigma -0.8324003178920438
Mean 1.3735450496022938
NEw mean 2.5197855346759854
sigma 2.6400840335445785
Mean 5.742252265027096
NEw mean 6.490651273404083
sigma 3.105839610760191
Mean 5.322733387568344
NEw mean 6.158851200058061
sigma 2.815370589638276


In [210]:
#magic numbers
good = parameters1[2:]
print(good) 
good = [[np.float64(0.10025691523156223), np.float64(-0.10754441565267893)], [np.float64(0.22740549074801997), np.float64(0.1071276402591709)], [np.float64(0.2125391014614699), np.float64(0.09476314939705265)]]


[[np.float64(0.10025691523156223), np.float64(-0.10754441565267893)], [np.float64(0.22740549074801997), np.float64(0.1071276402591709)], [np.float64(0.2125391014614699), np.float64(0.09476314939705265)]]


In [217]:
plt.plot(range(len(parameters1)+1),[0]+[pair[0] for pair in parameters1], label = "RPC from RPC 0", color = "blue")
plt.errorbar(range(len(parameters1)+1),[0]+[pair[0] for pair in parameters1], yerr = [0]+[abs(pair[1]) for pair in parameters1], fmt = "o", color = "blue")
plt.plot(range(1,len(parameters2)+2),[0]+[pair[0] for pair in parameters2], label = "RPC from RPC 1", color = "orange")
plt.errorbar(range(1,len(parameters2)+2),[0]+[pair[0] for pair in parameters2], yerr = [0]+[abs(pair[1]) for pair in parameters2], fmt = "o", color = "orange")

plt.xlabel("RPC pair")
plt.ylabel("Time of flight  [(25/32) ns / cm]")
plt.xlim(-1,6)
plt.title("Time of flight normalized per distance")
plt.legend()
plt.hlines(0, -1, 6, color = "black", linestyle = "--")
plt.show()

In [218]:
print(parameters1)


[[np.float64(0.25790911824685864), np.float64(0.5818488944295358)], [np.float64(0.06942782469835455), np.float64(0.5634127422432911)], [np.float64(2.17698945272121), np.float64(2.5044894876283283)], [np.float64(6.619902168271083), np.float64(2.91238011053787)], [np.float64(6.199631794023697), np.float64(2.617768586336547)]]


In [18]:
chi2 = []
for event in interesting:
    chi2.append(event[2][0].chi2)
chi2 = np.array(chi2)
hist, bins = np.histogram(chi2, bins=100, range=(0, 20))
plt.step(bins[:-1], hist, where='mid')
plt.show()

In [70]:
print(interesting[0][2][0].coordinates)
print(interesting[0][2][1].coordinates)

[[195.3125  103.59375  22.383     1.8    ]
 [196.09375 103.59375  22.383     3.     ]
 [197.65625 109.11875  28.3518   61.8    ]
 [200.      111.88125  34.3206  121.8    ]]
[[194.53125 103.59375  22.383     0.6    ]
 [195.3125  103.59375  22.383     1.8    ]
 [197.65625 109.11875  28.3518   61.8    ]
 [200.      111.88125  34.3206  121.8    ]]


In [48]:
hist, bins = np.histogram(reconstructor.cluster_size[0][0], bins = np.arange(0, 64, 1), density = True)
plt.step(bins[:-1], hist, label = r"Clusters corresponding to $-3 < \frac{\eta - \phi}{\sigma} < 3$", where='mid')
hist, bins = np.histogram(reconstructor.cluster_size[0][1], bins = np.arange(0, 64, 1), density = True)
plt.step(bins[:-1], hist, label = r"Clusters corresponding to $5.8 < \frac{\eta - \phi}{\sigma} < 6.5$", where='mid')
plt.yscale("log")
plt.xlabel("$\phi$ cluster size")
plt.ylabel("Density")
plt.title("Cluster size distribution of the primary and secondary peak")
plt.legend()
plt.show()

: 

In [127]:
fig, ax = plt.subplots(figsize=(10, 8))
TAnalyser.diffHists[5][4][24].plot1d(ax=ax, color="teal", lw=3, label='proANUBIS Data')
plt.show()

In [27]:
for rpc in range(6):
    hist, bins = np.histogram(reconstructor.collector[rpc], np.arange(-10, 10, 0.1), density=True)
    plt.plot(bins[:-1], hist, label=f"RPC {rpc}")
plt.vlines([-3,3], 0, max(hist), color='black', linestyles='dashed')
plt.text(-3, max(hist), "3$\sigma$ away", verticalalignment='bottom', horizontalalignment='right')
plt.xlabel("Time Difference ($\sigma_{tot}$)")
plt.ylabel("Distribution")
plt.title("$\eta - \phi$ time difference distribution")
plt.legend()
plt.show()

In [17]:
for rpc in range(6):
    values = []
    for cluster_size in reconstructor.collector[rpc].keys():
        values.append(np.mean(reconstructor.collector[rpc][cluster_size]))
    plt.scatter(list(reconstructor.collector[rpc].keys()), values, label=f'RPC {rpc}')

plt.xlabel('Cluster size')
plt.ylabel('Spread')
plt.legend(loc='upper left')
plt.xlim(0,10)
plt.ylim(0,10)
plt.show()

In [6]:
importlib.reload(proAnubis_Analysis_Tools)
TAnalyser = proAnubis_Analysis_Tools.Timing_Analyser(chunks[0],0)
for processedEvents, chunk in enumerate(chunks):
    TAnalyser.update_event(chunk, processedEvents)
    TAnalyser.readTDCTimeDiffs()

#TAnalyser.calculate_correction_stats()

residEta, residPhi = TAnalyser.Calculate_Residual_and_plot_TDC_Time_Diffs( 
                                                     pdf_filename='output/TDC_time_diffs.pdf', 
                                                     max_itr = 5)

print((residEta, residPhi))
outDict = {'totDiffs':TAnalyser.totDiffs,
            'nDiffs':TAnalyser.nDiffs,
            'diffHists':TAnalyser.diffHists} 


([[np.float64(2.081698275862068), np.float64(1.9524482758620676), np.float64(1.7356982758620685), np.float64(1.743948275862069), np.float64(1.4959482758620695), np.float64(1.2166982758620688), np.float64(1.0436982758620692), np.float64(1.0144482758620685), np.float64(1.2539482758620686), np.float64(1.5746982758620685), np.float64(0.7329482758620689), np.float64(0.44119827586206917), np.float64(0.7869482758620691), np.float64(-0.08605172413793112), np.float64(-0.01530172413793078), np.float64(-0.28830172413793087), np.float64(-0.35505172413793007), np.float64(-0.40305172413793067), np.float64(-0.6698017241379309), np.float64(-0.9178017241379313), np.float64(-1.1283017241379305), np.float64(-0.7013017241379295), np.float64(-1.1618017241379301), np.float64(-0.7535517241379304), np.float64(-1.51405172413793), np.float64(-2.2433017241379307), np.float64(-2.0038017241379285), np.float64(-2.0705517241379297), np.float64(-2.7623017241379304), np.float64(-2.9353017241379313), np.float64(-3.1083

In [16]:
print(len(residPhi[0]))

64


In [19]:
importlib.reload(VTools)
file_path = "data//interesting_events.pkl"
with open(file_path, 'rb') as f:
    data = pickle.load(f)
    print(len(data))
interesting_events = data
for event in interesting_events[10:11]:
   VTools.event_3d_plot(event, 0)

260


In [11]:
reconstructor = proAnubis_Analysis_Tools.Reconstructor(interesting_events, 0)
# populate_hits turns TDC bit wise information into their corresponding strips
reconstructor.populate_hits()
# This is optionnal, and requires the residual of eta and phi
    # make_cluster does temporal and spatial coincidence between the stips, and reconstruction is done
clusters = reconstructor.make_cluster()

chi2 = []
chi2_region = (0, 100)
for evt_num, cluster in enumerate(clusters):
    tracks = RTools.reconstruct_timed_Chi2_ByRPC(cluster, 3, -1)
    if tracks and len(tracks) > 1:
        chi2.append(tracks[1][4])
    print(evt_num)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181


KeyboardInterrupt: 

In [12]:
plt.figure(figsize=(10, 6))
max_eff = []
for RPC in range(6):
    if possible_reconstructions[RPC] == 0:
        efficiency = [0 for x in successful_reconstructions[RPC]]
    else:
        efficiency = [x / possible_reconstructions[RPC] for x in successful_reconstructions[RPC]]
    max_eff.append(efficiency[-1])
    plt.plot(tol, efficiency, label=f'RPC {RPC}')

plt.xlabel('Tolerance')
plt.ylabel('Efficiency')
plt.title('The efficiency of the second track reconstruction')
plt.legend()
plt.grid(True)
plt.show()
plt.close()
print("Efficiency Done")
print("Possible reconstructions", reconstructor.possible_reconstructions)
print(max_eff)

Efficiency Done
Possible reconstructions [0, 0, 0, 0, 0, 0]
[0.4318181818181818, 0.6, 0.6818181818181818, 0.4406779661016949, 0.35, 0.6071428571428571]


In [18]:
#plot chi2 distribution
import scipy.special as sp
k = np.mean(chi2)
plt.hist(chi2, bins = int(np.sqrt(8*len(chi2))), density = True)
x = np.array([i for i in range(0, 10)])
print(sp.gamma(k))
plt.plot(x, 2.4*x**(k/2-1)*np.exp(-x/2)/((2**(k/2))*sp.gamma(k/2)), label = f"Idealised χ2({round(k,2)}) distribution")#pdf of chi2
plt.title('χ2 distribution of the second track')
plt.xlabel('Chi2')
plt.legend()
plt.ylabel('Frequency')
plt.show()
print(chi2)

3.549413719172803
[np.float64(2.923268450788971), np.float64(1.5455626187848202), np.float64(1.3824512933971917), np.float64(0.2870659845529911), np.float64(6.057028098867192), np.float64(1.1170741127596033), np.float64(0.47158490850782714), np.float64(1.4008653273049967), np.float64(6.473761420102736), np.float64(1.6294728836774055), np.float64(1.5255178606337314), np.float64(0.4398499126206219), np.float64(3.4205517579528855), np.float64(2.100629585341305), np.float64(3.8330354225996115), np.float64(1.600927888787882), np.float64(4.449677296942161), np.float64(4.449677296942161), np.float64(4.449677296942161), np.float64(4.449677296942161), np.float64(4.449677296942161), np.float64(4.449677296942161), np.float64(4.6383741165254815), np.float64(4.6383741165254815), np.float64(4.6383741165254815), np.float64(4.6383741165254815), np.float64(4.6383741165254815), np.float64(4.6383741165254815), np.float64(5.698472209406913), np.float64(5.698472209406913), np.float64(5.698472209406913), np